<a href="https://colab.research.google.com/github/troncosofranco/ML-Notebooks/blob/main/Retail_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Contexto

El cliente no ha compartido un set de datos con información parcial sobre los puntos de ventas de sus diferentes productos.

La planta de galvanizado exporta la mayoría de sus productos via barco a su principal cliente en United Kindom. Sin embargo, las piezas son nuevamente revendidas a otros proveedores y/o consumidores finales.

El distribuidor europeo ha compartido datos antiguos sobre la venta de los mismos. El objetivo es explorar este gran set de datos para obtener insigths.

No tenemos toda la información completa y el precio de las unidades de tubería galvanizado deberia estar entre 1 y 5 U$d/ft.
No tenemos la descripción de los productos y no se tiene certeza sobre la consistencia de los datos.

#1. Librerias

In [ ]:
!pip install pyspark

In [ ]:
import numpy as np
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Análisis de ventas').getOrCreate()

#2. Carga Dataset

In [ ]:
df = spark.read.csv('online_retail_listing.csv',\
                            header = True,
                            sep = ';',
                            inferSchema = True)
# check the data
df.show(5)
df.printSchema()

+-------+---------+--------+---------------+--------+-----------+--------------+
|Invoice|StockCode|Quantity|    InvoiceDate|Price/ft|Customer ID|       Country|
+-------+---------+--------+---------------+--------+-----------+--------------+
| 489434|    85048|      12|1.12.2009 07:45|    6.95|      13085|United Kingdom|
| 489434|   79323P|      12|1.12.2009 07:45|    6.75|      13085|United Kingdom|
| 489434|   79323W|      12|1.12.2009 07:45|    6.75|      13085|United Kingdom|
| 489434|    22041|      48|1.12.2009 07:45|     2.1|      13085|United Kingdom|
| 489434|    21232|      24|1.12.2009 07:45|    1.25|      13085|United Kingdom|
+-------+---------+--------+---------------+--------+-----------+--------------+
only showing top 5 rows

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- Price/ft: double (nullable = true)
 |-- Customer ID: integer (nullable =

In [ ]:
from pyspark.sql.functions import col, sum
# Contar valores nulos en cada columna
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+-------+---------+--------+-----------+--------+-----------+-------+
|Invoice|StockCode|Quantity|InvoiceDate|Price/ft|Customer ID|Country|
+-------+---------+--------+-----------+--------+-----------+-------+
|      0|        0|       0|          0|       0|     236682|      0|
+-------+---------+--------+-----------+--------+-----------+-------+



In [ ]:
# Obtener el número de filas y columnas
num_filas = df.count()
num_columnas = len(df.columns)

# Imprimir las dimensiones del DataFrame
print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

El DataFrame tiene 1048575 filas y 7 columnas.


In [ ]:
df.describe().show()

+-------+------------------+------------------+-----------------+---------------+------------------+-----------------+-----------+
|summary|           Invoice|         StockCode|         Quantity|    InvoiceDate|          Price/ft|      Customer ID|    Country|
+-------+------------------+------------------+-----------------+---------------+------------------+-----------------+-----------+
|  count|           1048575|           1048575|          1048575|        1048575|           1048575|           811893|    1048575|
|   mean|  536825.471444893|29058.852051336133|9.957525212788784|           NULL| 4.627346139281089|15324.71226503985|       NULL|
| stddev|26251.184694935902| 18886.71005287471|133.5186919179497|           NULL|122.80241218014967|1697.033034188479|       NULL|
|    min|            489434|             10002|           -74215|1.02.2010 08:13|         -53594.36|            12346|  Australia|
|    max|           C580392|                 m|            74215|9.12.2010 20:01|  

Observamos que valores de Quanityt y Price/ft tienen valores negativos. En este caso asumiermos que fue un input incorrecto. Consideramos el valor absoluto para ambas columnas. Otra opción es filtrar el dataset solo para valores positivos.

In [ ]:
from pyspark.sql.functions import col, abs, max as spark_max, min as spark_min

# Supongamos que df es tu DataFrame y contiene las columnas 'Quantity', 'Price/ft', y 'Country'

# Aplicar la función de valor absoluto a 'Quantity' y 'Price/ft'
df = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Calcular la columna Quantity * Price/ft
df = df.withColumn("Value", col("Quantity") * col("Price/ft"))

# Agrupar por 'Country' y calcular el valor máximo y mínimo de 'Value'
aggregated_df = df.groupBy("Country").agg(
    spark_max("Value").alias("MaxValue"),
    spark_min("Value").alias("MinValue")
)

# Mostrar los resultados
aggregated_df.show()

+-----------+------------------+------------------+
|    Country|          MaxValue|          MinValue|
+-----------+------------------+------------------+
|     Sweden|            1188.0|              0.85|
|  Singapore|           3949.32|              2.34|
|    Germany|             876.0|               0.0|
|        RSA|           1435.29|               0.0|
|     France|           8322.12|               0.0|
|     Greece|             229.5|              0.85|
|    Belgium|           1508.65|               0.0|
|    Finland|             551.2|              0.78|
|      Malta|           1868.56|              1.06|
|    Nigeria|             79.05|              0.42|
|Unspecified|           1189.94|              0.19|
|      Italy|            660.86|              1.25|
|       EIRE|           2736.16|               0.0|
|  Lithuania|122.39999999999999|3.3600000000000003|
|     Norway|           6958.17|               0.0|
|      Spain|           1715.85|               0.0|
|    Denmark

Cuál fue la orden mínima y máxima por país?

In [ ]:
from pyspark.sql.functions import col, abs, max as spark_max, min as spark_min

# Aplicar la función de valor absoluto a 'Quantity' y 'Price/ft'
df_abs = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Calcular la columna Quantity * Price/ft
df_with_calculation = df_abs.withColumn("Value", col("Quantity") * col("Price/ft"))

# Agrupar por 'Country' y calcular el valor máximo y mínimo de 'Value'
aggregated_df = df_with_calculation.groupBy("Country").agg(
    spark_max("Value").alias("MaxValue"),
    spark_min("Value").alias("MinValue")
)

# Mostrar los resultados
aggregated_df.show()

+-----------+------------------+------------------+
|    Country|          MaxValue|          MinValue|
+-----------+------------------+------------------+
|     Sweden|            1188.0|              0.85|
|  Singapore|           3949.32|              2.34|
|    Germany|             876.0|               0.0|
|        RSA|           1435.29|               0.0|
|     France|           8322.12|               0.0|
|     Greece|             229.5|              0.85|
|    Belgium|           1508.65|               0.0|
|    Finland|             551.2|              0.78|
|      Malta|           1868.56|              1.06|
|    Nigeria|             79.05|              0.42|
|Unspecified|           1189.94|              0.19|
|      Italy|            660.86|              1.25|
|       EIRE|           2736.16|               0.0|
|  Lithuania|122.39999999999999|3.3600000000000003|
|     Norway|           6958.17|               0.0|
|      Spain|           1715.85|               0.0|
|    Denmark

In [ ]:
df.columns

['Invoice',
 'StockCode',
 'Quantity',
 'InvoiceDate',
 'Price/ft',
 'Customer ID',
 'Country']

In [ ]:
from pyspark.sql.functions import col, count, when, abs

# Aplicar la función de valor absoluto a 'Quantity' y 'Price/ft'
df_abs = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Contar el total de filas en el DataFrame
total_rows = df_abs.count()

# Calcular el porcentaje de valores nulos por columna
null_percentages = df_abs.select([
    (count(when(col(c).isNull(), c)) / total_rows * 100).alias(c)
    for c in df_abs.columns
])

# Mostrar los resultados
null_percentages.show()

+-------+---------+--------+-----------+--------+------------------+-------+-----+
|Invoice|StockCode|Quantity|InvoiceDate|Price/ft|       Customer ID|Country|Value|
+-------+---------+--------+-----------+--------+------------------+-------+-----+
|    0.0|      0.0|     0.0|        0.0|     0.0|22.571775981689434|    0.0|  0.0|
+-------+---------+--------+-----------+--------+------------------+-------+-----+



In [ ]:
from pyspark.sql.functions import col, abs, sum

# Aplicar la función de valor absoluto a 'Quantity' y 'Price/ft'
df_abs = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Calcular la columna Quantity * Price/ft
df_with_calculation = df_abs.withColumn("Value", col("Quantity") * col("Price/ft"))

# Agrupar por 'Country' y sumar los valores calculados
aggregated_df = df_with_calculation.groupBy("Country").agg(
    sum("Value").alias("TotalSales")
)

# Ordenar los resultados en orden descendente por la suma calculada
sorted_df = aggregated_df.orderBy(col("TotalSales").desc())

# Mostrar los resultados
sorted_df.show()

+---------------+--------------------+
|        Country|          TotalSales|
+---------------+--------------------+
| United Kingdom|1.8677388427002504E7|
|           EIRE|   707042.0400000089|
|    Netherlands|   548211.7100000011|
|        Germany|  437164.76099999994|
|         France|   380893.9199999999|
|      Australia|   172807.1499999995|
|          Spain|  126136.59000000003|
|    Switzerland|  102293.82000000015|
|         Sweden|   95522.01999999987|
|         Norway|   78190.50999999998|
|        Denmark|   73814.39000000001|
|        Belgium|   66824.94000000006|
|       Portugal|   61519.00000000006|
|          Japan|  50383.270000000004|
|Channel Islands|  48337.260000000024|
|      Singapore|  37475.959999999985|
|          Italy|   34221.04000000002|
|      Hong Kong|  33749.340000000026|
|        Finland|  30334.549999999977|
|         Cyprus|  25797.520000000015|
+---------------+--------------------+
only showing top 20 rows



Promedio de ticket por país

In [ ]:
# Aplicar la función de valor absoluto a 'Quantity' y 'Price/ft'
df_abs = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Agrupar por 'Country' y calcular el promedio de 'Price/ft' y 'Quantity'
aggregated_df = df_abs.groupBy("Country").agg(
    round(avg("Price/ft"), 1).alias("AveragePricePerFt"),
    round(avg("Quantity"), 1).alias("AverageQuantity")
)

# Ordenar los resultados en orden descendente por el promedio de 'AveragePricePerFt'
sorted_df = aggregated_df.orderBy(col("AveragePricePerFt").desc())

# Mostrar los resultados
sorted_df.show()

+--------------------+-----------------+---------------+
|             Country|AveragePricePerFt|AverageQuantity|
+--------------------+-----------------+---------------+
|           Singapore|             73.6|           20.2|
|           Hong Kong|             57.6|           19.5|
|              Norway|             29.6|           18.1|
|               Malta|             22.0|            8.5|
|                 RSA|             19.8|           11.7|
|                EIRE|              7.1|           19.7|
|            Portugal|              6.6|           11.0|
|              Sweden|              6.4|           65.6|
|             Lebanon|              6.2|            7.9|
|               Spain|              5.5|           15.0|
|               Italy|              5.5|           10.3|
|              Cyprus|              5.4|            9.6|
|     Channel Islands|              5.4|           13.5|
|United Arab Emirates|              5.1|           15.8|
|             Finland|         

In [ ]:
from pyspark.sql.functions import col, abs, countDistinct

# Aplicar la función de valor absoluto a las columnas numéricas (si fuera necesario)
df_abs = df.withColumn("Quantity", abs(col("Quantity"))).withColumn("Price/ft", abs(col("Price/ft")))

# Agrupar por 'Country' y contar los valores distintos de 'Customer ID'
customer_counts = df_abs.groupBy("Country").agg(
    countDistinct("Customer ID").alias("DifferentCustomerCount")
)

# Ordenar los resultados en orden descendente por la cuenta de 'Customer ID'
sorted_customer_counts = customer_counts.orderBy(col("DifferentCustomerCount").desc())

# Mostrar los resultados
sorted_customer_counts.show()

+---------------+----------------------+
|        Country|DifferentCustomerCount|
+---------------+----------------------+
| United Kingdom|                  5397|
|        Germany|                   106|
|         France|                    94|
|          Spain|                    40|
|        Belgium|                    29|
|       Portugal|                    24|
|    Netherlands|                    23|
|    Switzerland|                    22|
|         Sweden|                    19|
|          Italy|                    17|
|        Finland|                    15|
|      Australia|                    15|
|Channel Islands|                    14|
|         Norway|                    13|
|        Austria|                    13|
|        Denmark|                    11|
|         Cyprus|                    11|
|          Japan|                    10|
|            USA|                     9|
|    Unspecified|                     7|
+---------------+----------------------+
only showing top

Cuál es el producto que en promedio se ha vendido más caro?

In [ ]:
from pyspark.sql.functions import col, abs, avg, round

# Aplicar la función de valor absoluto a 'Price/ft'
df_abs = df.withColumn("Price/ft", abs(col("Price/ft")))

# Agrupar por 'StockCode' y calcular el promedio de 'Price/ft'
aggregated_df = df_abs.groupBy("StockCode").agg(
    round(avg("Price/ft"), 1).alias("AveragePricePerFt")
)

# Ordenar los resultados en orden descendente por el promedio calculado (opcional)
sorted_df = aggregated_df.orderBy(col("AveragePricePerFt").desc())

# Mostrar los resultados
sorted_df.show()

+------------+-----------------+
|   StockCode|AveragePricePerFt|
+------------+-----------------+
|           B|          28289.7|
|   AMAZONFEE|           7301.4|
|           M|            532.1|
|        CRUK|            455.5|
|BANK CHARGES|            362.4|
|     ADJUST2|            243.7|
|         DOT|            212.8|
|       22656|            188.3|
|       22655|            177.4|
|      ADJUST|            163.6|
|       22828|            154.2|
|       22827|            149.9|
|       22826|            115.4|
|       22823|            105.8|
|       21760|             98.3|
|       21769|             76.0|
|       85070|             74.2|
|           D|             72.9|
|       22929|             64.3|
|           S|             61.0|
+------------+-----------------+
only showing top 20 rows

